In [ ]:
import numpy as np
import pymc as pm
import pytensor as pt
import scipy
import arviz as az
import pymc.backends as pmbackends
import sys

sys.setrecursionlimit(1000000) 

In [ ]:
u_t = np.loadtxt("demo_files/u_t.txt", dtype=int)
#u_t = u_t1.tolist()
resp = np.loadtxt("demo_files/resp.txt", dtype=int)
#resp = resp1.tolist()


def calc_ned(omega2, omega3, kappa,u_t):
    u_t = pt.tensor.as_tensor_variable(u_t,dtype="int")

    mu2_t2 = 0
    mu3_t2 = 1
   
    pi2_t2 = 10
    pi3_t2 = 1
    da2_t1 = 0

    mu_hat = pt.tensor.zeros(100)
    
    for trial in range(100):
        
        mu_hat2_t1 = mu2_t2
        mu_hat1_t1 = 1 / ( 1 + pt.tensor.exp(-mu_hat2_t1))
        pi_hat1_t1 = 1 / (mu_hat1_t1 * (1 - mu_hat1_t1))    
        
        mu1_t1_scalar = u_t[trial]
        da1_t1 = mu1_t1_scalar - mu_hat1_t1
        
        v2_t1 = pt.tensor.exp(kappa * mu3_t2 + omega2)
        pi_hat2_t1 = 1/((1/pi2_t2) + v2_t1 )
        pi2_t1 = pi_hat2_t1 + 1 / pi_hat1_t1
        mu2_t1 = mu_hat2_t1 + (1 / pi2_t1) * da1_t1
        da2_t1 = (1 / pi2_t1 + ((mu2_t1 - mu_hat2_t1) ** 2)) * pi_hat2_t1 - 1

        mu_hat3_t1 = mu3_t2
        pi_hat3_t1 = 1 / ((1/pi3_t2) + pt.tensor.exp(omega3))          
        w2_t1 = v2_t1 * pi_hat2_t1

        pi3_t1 = pi_hat3_t1 + 0.5 * (kappa ** 2) * w2_t1 * ((w2_t1 + (2 * w2_t1 - 1) * da2_t1))
        mu3_t1 = mu_hat3_t1 + 0.5 * (1 / pi3_t1 ) * kappa * w2_t1 * da2_t1

        mu2_t2 = mu2_t1
        mu3_t2 = mu3_t1
        
        pi2_t2 = pi2_t1
        pi3_t2 = pi3_t1
         
        mu_hat=pt.tensor.set_subtensor(mu_hat[trial],mu_hat1_t1)
                

    return mu_hat

In [ ]:
with pm.Model() as model:
    omega2 = pm.Normal('omega2', mu=-3, sigma=16, initval=-3.7)
    omega3 = pm.Normal('omega3',mu=-6,sigma=16, initval=-5)
    kappa = pm.HalfNormal('kappa', sigma=16, initval=0.2)

    action_prob = calc_ned(omega2,omega3,kappa,u_t)
    like = pm.Bernoulli(name="like", p=action_prob, observed=resp)
    try:
        trace = pm.sample(draws=500,tune=500,progressbar=True,chains=4)
    except pm.SamplingError as error:
        print(error)
        model.debug()

In [ ]:
az.plot_trace(data=trace)

In [ ]:
u_t = np.loadtxt("demo_files/u_t.txt", dtype=int)
resp = np.loadtxt("demo_files/resp.txt", dtype=int)


def calc_np(omega, theta, kappa,u_t):

    mu1_t2 = 0
    mu2_t2 = 0
    mu3_t2 = 1
    pi1_t2 = np.inf
    pi2_t2 = 10
    pi3_t2 = 1
    da2_t1 = 0
    da3_t1 = 0
    

    mu_hat = np.zeros(320,)      

    pi2_t1 = np.zeros(320,)
     #1ST LVL 
  
    for trial in range(320):

        mu_hat2_t1 = mu2_t2  #mu_hat[trial,1] = mu[trial-1,1]
        
        ##----------------------------1ST LVL---------------------------------------
        
        mu_hat1_t1 = 1/(1+np.exp(-mu_hat2_t1)) # mu_hat[trial,0] = _sgm(p_dict['ka'][0] * mu_hat[trial,1], 1)
        pi_hat1_t1 = 1 / (mu_hat1_t1 * (1 - mu_hat1_t1)) #pi_hat[trial,0] = 1 / (mu_hat[trial,0] * (1-mu_hat[trial,0]))

        pi1_t1 = np.inf  ##pi[trial,0] = np.inf
        mu1_t1_scalar = u_t[trial] ##mu[trial,0] = u[trial]
        
        da1_t1 = mu1_t1_scalar - mu_hat1_t1
        
        ##----------------------------2ND LVL---------------------------------------
        
        pi_hat2_t1 = (pi2_t2) ** -1 + np.exp(kappa * mu3_t2 + omega) ** -1##pi_hat[trial,lvl] = (pi[trial-1,lvl]**-1 + t[trial] * np.exp(p_dict['ka'][lvl] *  mu[trial-1, lvl+1] +p_dict['om'][lvl]))**-1
        
        pi2_t1 = pi_hat2_t1 + 1 / pi_hat1_t1   ##pi[trial,1] = pi_hat[trial,1] + p_dict['ka'][0]**2 / pi_hat[trial,0]
        mu2_t1 = mu_hat2_t1 + (1 / pi2_t1) * da1_t1##mu[trial,1] = mu_hat[trial,1] + p_dict['ka'][0] / pi[trial,1] * da[trial,0]

        ##----------------------------3RD LVL---------------------------------------

        mu_hat3_t1 = mu3_t2 ##mu_hat[trial,lvl] = mu[trial-1,lvl] + (t[trial]*p_dict['rho'][lvl])
        
        pi_hat3_t1 = (pi2_t2) ** -1 + theta ** -1  ##pi_hat[trial,l-1] = (pi[trial-1,l-1]**-1 + t[trial] * p_dict['th'])**-1
        v3_t1 = theta ##v[trial,l-1] = t[trial] * p_dict['th']
        v2_t1 = np.exp(kappa * mu3_t2 + omega)  ## v[trial,l-2] = t[trial] * np.exp(p_dict['ka'][l-2] * mu[trial-1, l-1] + p_dict['om'][l-2])
        w2_t1 = v2_t1 * pi_hat2_t1  ##w[trial,l-2] = v[trial,l-2] * pi_hat[trial,l-2]
        
        ##--------------------------LVL=3----------------------------------------------------
        
        pi3_t1 = pi_hat3_t1 + 0.5 * (kappa**2) * w2_t1 * ((w2_t1 + (2 * w2_t1 - 1) * da2_t1))
        mu3_t1 = mu_hat3_t1 + 0.5 * (pi3_t1**-1) * kappa * w2_t1 * da2_t1
        da3_t1 = (pi3_t1**-1) + ((mu3_t1 - mu_hat3_t1)**2) * pi_hat3_t1 - 1
        
        ##----------------------------LVL=2--------------------------------------------------
        pi2_t1 = pi_hat2_t1 
        mu2_t1 = mu_hat2_t1 
        da2_t1 = (pi2_t1**-1) + ((mu2_t1 - mu_hat2_t1)**2) * pi_hat2_t1 - 1
        ##----------------------------------------------------------           

        mu_hat[trial] = mu_hat1_t1
        
    
    return mu_hat


In [ ]:
u_t = [0,1,0,1,0,]
resp = [1,0,1,1,1]

def calc_n(omega, theta, kappa, u_t):
    mu1_t2 = 0
    mu2_t2 = 0
    mu3_t2 = 1
    #pi1_t2 = np.inf
    pi2_t2 = 10
    pi3_t2 = 1
    da2_t1 = 0
    da3_t1 = 0
    
    mu_hat = np.zeros(320)
    
    for trial in range(320):
        mu_hat2_t1 = mu2_t2
        mu_hat1_t1 = 1 / (1 + np.exp(-mu_hat2_t1))
        pi_hat1_t1 = 1 / (mu_hat1_t1 * (1 - mu_hat1_t1))
        
        #pi1_t1 = np.inf
        mu1_t1_scalar = u_t[trial]
       
        da1_t1 = mu1_t1_scalar - mu_hat1_t1
        
        pi_hat2_t1 = 1/((1/pi2_t2) + np.exp(kappa * mu3_t2 + omega) )
        pi2_t1 = pi_hat2_t1 + 1 / pi_hat1_t1
        mu2_t1 = mu_hat2_t1 + (1 / pi2_t1) * da1_t1
        da2_t1 = (1 / pi2_t1 + ((mu2_t1 - mu_hat2_t1) ** 2)) * pi_hat2_t1 - 1
        
        mu_hat3_t1 = mu3_t2
        pi_hat3_t1 = 1 / ((1/pi3_t2) + theta) 
        v3_t1 = theta
        v2_t1 = np.exp(kappa * mu3_t2 + omega)
        w2_t1 = v2_t1 * pi_hat2_t1
        
        pi3_t1 = pi_hat3_t1 + 0.5 * (kappa ** 2) * w2_t1 * ((w2_t1 + (2 * w2_t1 - 1) * da2_t1))
        mu3_t1 = mu_hat3_t1 + 0.5 * (pi3_t1 ** -1) * kappa * w2_t1 * da2_t1
        da3_t1 = (pi3_t1 ** -1) + ((mu3_t1 - mu_hat3_t1) ** 2) * pi_hat3_t1 - 1
        
        
        
        mu_hat1_t1 = 1 / (1 + np.exp(-mu_hat2_t1))
        
        mu1_t2 = mu1_t1_scalar
        mu2_t2 = mu2_t1
        mu3_t2 = mu3_t1
        #pi1_t2 = pi1_t1
        pi2_t2 = pi2_t1
        pi3_t2 = pi3_t1
        
        mu_hat[trial] = mu_hat1_t1
        
    return pt.tensor.tensor(mu_hat)

In [ ]:

s = calc_n(-6.115423812300319,0.002209784784040789,0.501248554597270,u_t)

